In [1]:
from __future__ import print_function
import os
data_path = ['data']

import pandas as pd
import numpy as np

# Import the data using the file path
filepathTrain = os.sep.join(['train.csv'])
trainData = pd.read_csv(filepathTrain)


In [2]:
#getting the target and the trainings data
x = trainData.iloc[:,trainData.columns != 'Activity']  
y = trainData.iloc[:,0]    #target column

from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

#tree classifier to determine the most impactful features as to not having to use all features/overfitting
#because trees determine the highest impact of a feature it looked as a good choice to use to determine the most impactful
model = ExtraTreesClassifier()
model.fit(x,y)
feat_importances = pd.Series(model.feature_importances_, index=x.columns)
results = feat_importances.nlargest(500)
#print(results.index)

C:\Users\chris\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [3]:
#select the top important features
trainDataFeats = trainData[results.index]

from sklearn import model_selection
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import random


#knn = KNeighborsClassifier(n_neighbors=3)

#iniatializing the randomforest with mostly defaults except n_estimators at 1000 and random state for reproducibility
clf = RandomForestClassifier(1000, random_state = random.seed(26))

#another way to measure the accuracy with cross validation
print(np.mean(cross_val_score(clf, trainDataFeats, y, cv=10)))


#fitting and predicting of the KNN algorithm
#knn = knn.fit(trainDataFeats, y)
#y_pred = knn.predict(trainDataFeats)

#fitting and predicting of the random forest
clf = clf.fit(trainDataFeats, y)
y_predclf = clf.predict(trainDataFeats)

0.8056516706489172


In [4]:
#this is code to test different input parameters for the random forest algorithm. 
#i commented this because it takes more than 30 mins to run but you're welcome to uncomment and run

#from sklearn.model_selection import GridSearchCV

#param_grid = {
               # 'criterion' : ["gini", "entropy"],
                 #'n_estimators': [1,10,100,500, 1000],
                # 'max_depth': [7, 9],
                #'min_samples_split': [2,10],
                #'max_features' : ['auto','log2' ]
             #}


#grid_clf = GridSearchCV(clf, param_grid, cv=10)
#grid_clf.fit(trainDataFeats, y)

#print(grid_clf.best_estimator_)
#print(grid_clf.best_params_)
#print(grid_clf.cv_results_)


# above gridsearchcv resulted in the following results when i ran it
#'criterion': 'entropy', 'max_depth': 9, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 1000}


#using mostly defaults though except n_estimators 1000 still give best results as can been seen above...

In [5]:
#accuracy of the predictions, stays on 1.0 mostly with the random forest, sounds like overfitting still..
def accuracy(real, predict):
    return sum(y == y_predclf) / float(real.shape[0])
print(accuracy(y, y_predclf))


1.0


In [6]:
#knn Probability, old code which i started with which uses the KNN algorithm but this resulted in scores of like 6 on kaggle
#and thus was not sufficient enough to use..
#filepathTest = os.sep.join(['test.csv'])
#testDataProb = pd.read_csv(filepathTest)
#testDataFeatsProb = testDataProb[results.index]


#y_prob2 = knn.predict_proba(testDataFeatsProb)[:,1]
#testDataProb.insert(0, "Probability", y_prob2)
#resultsdf = pd.DataFrame()
#resultsdf.insert(0, "PredictedProbability", y_prob2clf) 
#resultsdf.insert(0, "MoleculeId", range(1,2502))
#export_csv = resultsdf.to_csv (r'biologicalResponseknn.csv', index = None, header=True)

In [7]:
#random forest probability results
#ive read that the random forest tree is a good algorithm because it gets rid of the variance introduced by some very
#high impacting features and because i've already chosen the top 500 features to reduce overfitting
#it seemed like a good idea to use and so far the results have been way better than KNN
#u can find some example knn predicting in the document, this resulted in a kaggle score of 6+

filepathTest = os.sep.join(['test.csv'])
testDataProbClf = pd.read_csv(filepathTest)
testDataFeatsClf = testDataProbClf[results.index]

#predict the test data just using the probabilities for an 1 or true (hence the :,1) because we dont care about both chances
y_prob2clf = clf.predict_proba(testDataFeatsClf)[:,1]
#make new dataframe and fill it with the data and export it as csv for kaggle
resultsdf = pd.DataFrame()
resultsdf.insert(0, "PredictedProbability", y_prob2clf) 
resultsdf.insert(0, "MoleculeId", range(1,2502))
export_csv = resultsdf.to_csv (r'biologicalResponseRF.csv', index = None, header=True)
